In [2]:
import tensorflow as tf
import numpy as np
from cvnn.layers import Convolutional
from pdb import set_trace
import sys
from scipy import signal
from scipy import linalg

Both TF and NP results calculate fft the same way

In [2]:
aaa = np.linspace(1.0, 10000.0, 10000)
x = aaa + 1j * aaa
x_tensor = tf.convert_to_tensor(x)

tf_fft = tf.signal.fft(x_tensor)
np_fft = np.fft.fft(x)

print(tf_fft.dtype)
print(np.all(tf_fft.numpy() == np_fft))  # Results are not exactly the same (but fair enough)
print(tf_fft.numpy()[:10])
print(np_fft[:10])
print(tf_fft.numpy() == np_fft)
print((tf_fft.numpy() - np_fft)[1])

<dtype: 'complex128'>
False
[ 50005000.        +50005000.j
 -15920493.78559075+15910493.78559075j
  -7962746.10739719 +7952746.10739719j
  -5310163.19893343 +5300163.19893342j
  -3983871.48290206 +3973871.48290206j
  -3188096.2438436  +3178096.2438436j
  -2657579.24327153 +2647579.24327153j
  -2278638.37897732 +2268638.37897732j
  -1994432.59985672 +1984432.59985672j
  -1773383.54418512 +1763383.54418512j]
[ 50005000.        +50005000.j
 -15920493.78559075+15910493.78559075j
  -7962746.10739719 +7952746.10739719j
  -5310163.19893342 +5300163.19893342j
  -3983871.48290206 +3973871.48290206j
  -3188096.2438436  +3178096.2438436j
  -2657579.24327152 +2647579.24327152j
  -2278638.37897732 +2268638.37897732j
  -1994432.59985672 +1984432.59985672j
  -1773383.54418512 +1763383.54418512j]
[ True False  True ... False False False]
(1.862645149230957e-09+0j)


## Testing on 1D

In [3]:
b = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
c = [1, 0, 1]

b_pad = tf.cast(tf.pad(b, tf.constant([[0, 2]])), tf.complex64)  # Full padding
I = tf.signal.fft(tf.cast(b_pad, tf.complex64))
paddings = tf.constant([[0, 9]])
c_pad = tf.cast(tf.pad(c, paddings), tf.complex64)
C = tf.signal.fft(c_pad)
F = tf.math.multiply(I, C)
f = tf.signal.ifft(F)
f_real = tf.cast(f, tf.int32)

# print("std_out: " + str(std_out))
print("f_real:  \t" + str(f_real.numpy()))
print("convolve:\t" + str(np.convolve(b, c)))

manual_conv = []
for i in range(len(b)-len(c)+1):
    manual_conv.append(np.sum(tf.math.multiply(c, b[i:i+3]).numpy()))
print("Manual nn conv: " + str(manual_conv))

c.reverse()
manual_conv = []
for i in range(len(b)-len(c)+1):
    manual_conv.append(np.sum(tf.math.multiply(c, b[i:i+3]).numpy()))
print("Manual fft conv:" + str(manual_conv))

f_real:  	[ 0  1  2  4  6  8 10 12 14 16  8  9]
convolve:	[ 0  1  2  4  6  8 10 12 14 16  8  9]
Manual nn conv: [2, 4, 6, 8, 10, 12, 14, 16]
Manual fft conv:[2, 4, 6, 8, 10, 12, 14, 16]


## Testing on 2D

In [3]:
np.set_printoptions(suppress=True)
img2 = np.array([
    [10, 10, 10, 0, 0, 0],
    [10, 10, 10, 0, 0, 0],
    [10, 10, 10, 0, 0, 0],
    [10, 10, 10, 0, 0, 0],
    [10, 10, 10, 0, 0, 0],
    [10, 10, 10, 0, 0, 0]
]).astype(np.float64)
k = np.array([
        [1., 0., -1.],
        [1., 0., -1.],
        [1., 0., -1.]
    ]).astype(np.float64)
mode = 'full'

conv = Convolutional(1, (3, 3), (6, 6, 1), padding=2, input_dtype=np.float32)
conv.kernels = []
conv.kernels.append(tf.reshape(tf.cast(tf.Variable(k, name="kernel" + str(0) + "_f" + str(0)), dtype=np.float32),
                               (3, 3, 1)))
std_out = conv([img2])[..., 0]
print("manual_conv: \n" + str(std_out.numpy()))

img_tf = tf.constant(tf.reshape(img2, (1, 6, 6, 1)), dtype=tf.float64)
k_tf = tf.constant(tf.reshape(k, (3, 3, 1, 1)), dtype=tf.float64)
conv_tf = tf.nn.conv2d(img_tf, k_tf, strides=[1, 1], padding="SAME")[0, ..., 0]
print("tf_nn_conv2d: \n" + str(np.around(conv_tf.numpy())))
# set_trace()

img2_pad = tf.pad(img2.astype(np.float64), tf.constant([[0, 2], [0, 2]]))
k_pad = tf.pad(k, tf.constant([[0, 5], [0, 5]]))
I = tf.signal.fft2d(tf.cast(img2_pad, tf.complex128))
print(I)
K = tf.signal.fft2d(tf.cast(k_pad, tf.complex128))
F = tf.math.multiply(I, K)
f = tf.signal.ifft2d(F)
f_real = tf.cast(f, tf.int32)
print("manual_fft_conv: " + str(f_real))

np_fft_conv = np.array(signal.fftconvolve(img2, k, mode=mode) , np.int32)
print("sp_fft_conv_" + mode + ":\n" + str(np_fft_conv))

np_conv = np.array(signal.convolve2d(img2 , k, mode), np.int32)
print("sp_conv2d" + mode + ":\n" + str(np_conv))

# Check numpy implementation
I = np.fft.fft2(img2_pad)
K = np.fft.fft2(tf.pad(k, tf.constant([[0, 5], [0, 5]])))
F = np.multiply(I, K)
f = np.fft.ifft2(F)
print("np_fft_conv: \n" + str(np.round(f.astype(np.float32))))



2020-11-16 18:55:05,977 — WARNING - layers::_verify_inputs line 569 — input dtype (<dtype: 'float64'>) not what expected (<class 'numpy.float32'>). Attempting cast...


manual_conv: 
[[[-10. -10.   0.  10.  10.   0.   0.   0.]
  [-20. -20.   0.  20.  20.   0.   0.   0.]
  [-30. -30.   0.  30.  30.   0.   0.   0.]
  [-30. -30.   0.  30.  30.   0.   0.   0.]
  [-30. -30.   0.  30.  30.   0.   0.   0.]
  [-30. -30.   0.  30.  30.   0.   0.   0.]
  [-20. -20.   0.  20.  20.   0.   0.   0.]
  [-10. -10.   0.  10.  10.   0.   0.   0.]]]
tf_nn_conv2d: 
[[-20.   0.  20.  20.   0.   0.]
 [-30.   0.  30.  30.   0.   0.]
 [-30.   0.  30.  30.   0.   0.]
 [-30.   0.  30.  30.   0.   0.]
 [-30.   0.  30.  30.   0.   0.]
 [-20.   0.  20.  20.   0.   0.]]
tf.Tensor(
[[180.          +0.j         102.42640687-102.42640687j
   -0.         -60.j          17.57359313 +17.57359313j
   60.          +0.j          17.57359313 -17.57359313j
    0.         +60.j         102.42640687+102.42640687j]
 [-21.21320344 -51.21320344j -41.21320344 -17.07106781j
  -17.07106781  +7.07106781j   2.92893219  -7.07106781j
   -7.07106781 -17.07106781j  -7.07106781  -2.92893219j
   17.07106781

C:\Users\NEGU93\.conda\envs\tf-2-gpu\lib\site-packages\ipykernel_launcher.py:51: ComplexWarning: Casting complex values to real discards the imaginary part


There are 2 results here and they are:

- $(x*v)(n) = \sum x(m) \, v(m)$
- $(x*v)(n) = \sum x(m) \, v(n-m)$

## StackOverflow Example

https://stackoverflow.com/questions/40703751/using-fourier-transforms-to-do-convolution

In [5]:
x = np.array([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, 3, 0], [0, 0, 0, 1]])
y = np.array([[4, 5], [3, 4]])

print("conv:\n",  signal.convolve2d(x, y, 'full'))

s1 = np.array(x.shape)
s2 = np.array(y.shape)
size = s1 + s2 - 1      # Full padding size = (5, 5)
fsize = 2 ** np.ceil(np.log2(size)).astype(int)    # I do this to have a 2^n size to make fft faster
# fsize = (8, 8)
fslice = tuple([slice(0, int(sz)) for sz in size])  
# slice to get the values later ([0:5], [0:5])

new_x = np.fft.fft2(x, fsize)
new_y = np.fft.fft2(y, fsize)
result = np.fft.ifft2(new_x*new_y)[fslice].copy()

print("manual fft method:\n", np.array(result.real, np.int32))

print("fft:\n" , np.array(signal.fftconvolve(x, y), np.int32))

conv:
 [[ 4  5  0  0  0]
 [ 3  0 -5  0  0]
 [ 0 -3  8 15  0]
 [ 0  0  9 16  5]
 [ 0  0  0  3  4]]
manual fft method:
 [[ 4  5  0  0  0]
 [ 2  0 -5  0  0]
 [ 0 -3  8 15  0]
 [ 0  0  9 16  5]
 [ 0  0  0  3  4]]
fft:
 [[ 3  5  0  0  0]
 [ 2  0 -5  0  0]
 [ 0 -2  8 15  0]
 [ 0  0  9 16  4]
 [ 0  0  0  2  4]]


## Complex Conv

In [6]:
img2 = np.array([
    [10, 10, 10, 0, 0, 0],
    [10, 10, 10, 0, 0, 0],
    [10, 10, 10, 0, 0, 0],
    [10, 10, 10, 0, 0, 0],
    [10, 10, 10, 0, 0, 0],
    [10, 10, 10, 0, 0, 0]
]).astype(np.float64)
k = np.array([
        [1., 0., -1.],
        [1., 0., -1.],
        [1., 0., -1.]
    ]).astype(np.float64)
mode = 'full'

c_img = tf.Variable(tf.complex(img2, img2))
c_k = tf.Variable(tf.complex(k, np.zeros(k.shape)))
conv_tf = tf.nn.conv2d(c_img, c_k, strides=[1, 1], padding="SAME")[0, ..., 0]
print(conv_tf)

NotFoundError: Could not find valid device for node.
Node:{{node Conv2D}}
All kernels registered for op Conv2D :
  device='CPU'; T in [DT_HALF]
  device='CPU'; T in [DT_FLOAT]
  device='CPU'; T in [DT_DOUBLE]
  device='CPU'; T in [DT_INT32]
  device='GPU'; T in [DT_HALF]
  device='GPU'; T in [DT_FLOAT]
  device='GPU'; T in [DT_DOUBLE]
  device='GPU'; T in [DT_INT32]
 [Op:Conv2D]

https://stackoverflow.com/questions/47577458/complex-convolution-in-tensorflow

## Tensorflow fftconv2d

In [7]:
import tensorflow as tf

def _centered(arr, newshape):
    # Return the center newshape portion of the array.
    currshape = tf.shape(arr)[-2:]
    startind = (currshape - newshape) // 2
    endind = startind + newshape
    return arr[..., startind[0]:endind[0], startind[1]:endind[1]]

def fftconv(in1, in2, mode="full"):
    mode = mode.lower()
    # Reorder channels to come second (needed for fft)
    in1 = tf.transpose(in1, perm=[0, 3, 1, 2])
    in2 = tf.transpose(in2, perm=[0, 3, 1, 2])

    # Extract shapes
    s1 = tf.convert_to_tensor(tf.shape(in1)[-2:])
    s2 = tf.convert_to_tensor(tf.shape(in2)[-2:])
    shape = s1 + s2 - 1

    # Compute convolution in fourier space
    sp1 = tf.spectral.rfft2d(in1, shape)
    sp2 = tf.spectral.rfft2d(in2, shape)
    ret = tf.spectral.irfft2d(sp1 * sp2, shape)

    # Crop according to mode
    if mode == "full":
        cropped = ret
    elif mode == "same":
        cropped = _centered(ret, s1)
    elif mode == "valid":
        cropped = _centered(ret, s1 - s2 + 1)
    else:
        raise ValueError("Acceptable mode flags are 'valid',"
                         " 'same', or 'full'.")

    # Reorder channels to last
    result = tf.transpose(cropped, perm=[0, 2, 3, 1])
    return result

In [8]:
onv_tf = fftconv(img_tf, k_tf, mode="SAME")[0, ..., 0]

AttributeError: module 'tensorflow' has no attribute 'spectral'